In [1]:
import torch
import os 
import sklearn 
import pandas as pd 
import numpy as np
from carla.data.catalog import CsvCatalog

/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
#TODO How to get access to test set from CARLA ? 
name='credit'
dataset = pd.read_csv(f'../data/{name}/{name}.csv')
            #TODO Better way for defining continous varaibles ?
continuous_wachter = dataset.drop(columns=['label']).columns
dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method='Identity')
dataset._df_test

,label,x1,x2,x3,x4,x6,x5,x7
1199,1.0,0.0,4.306475,0.087260,-1.376046,1.561485,-5.913621,2.470089
7904,1.0,0.0,-15.775737,-0.122886,-2.943257,-0.376813,-9.299610,-1.925437
7129,0.0,1.0,14.553110,0.277950,0.205990,-0.140736,2.985180,-3.622410
9092,1.0,0.0,-7.288998,-0.136467,-1.786576,0.856396,-6.075737,4.244439
8158,1.0,0.0,0.974857,-0.081596,3.588600,-2.664992,2.937239,-6.285523
...,...,...,...,...,...,...,...,...
8651,1.0,0.0,-7.321844,-0.371335,-2.246688,-3.866540,-4.158503,-13.891335
289,0.0,1.0,-6.419160,0.001437,1.523204,0.343382,3.340111,-9.342580
8530,0.0,1.0,-15.257133,-0.133623,4.657714,0.262945,4.614582,-3.449739
9118,0.0,1.0,28.465398,0.149740,-1.107442,4.990534,7.755683,-2.158499


In [3]:
model_name=['0']
accuracy=['0']
precision=['0']
recall=['0']
df=pd.DataFrame([])
for name in os.listdir('./Model/'):
    full_name=name
    model=torch.load(f'./Model/{name}')
    name=name.split('_')[1]
    name=name.replace('.pth','')
    dataset = pd.read_csv(f'../data/{name}/{name}.csv')
    continuous_wachter = dataset.drop(columns=['label']).columns
    dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method='Identity')
    dataset=dataset._df_test
    data_x, data_y=dataset.drop(columns=['label']), dataset['label']
    data_pred= model.predict(data_x)
    model_name.append(full_name)
    print(data_pred.shape)
    print(data_y.shape)
    if len(data_pred.shape)>1:
        data_pred= np.argmax(data_pred, axis=1)

    accuracy.append(sklearn.metrics.accuracy_score(data_y,data_pred))
    precision.append(sklearn.metrics.precision_score(data_y,data_pred))
    recall.append(sklearn.metrics.recall_score(data_y,data_pred))
df['model_name']=model_name
df['accuracy']=accuracy
df['precision']=precision
df['recall']=recall
df

(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


,model_name,accuracy,precision,recall
0,0,0,0,0
1,Forest_credit.pth,0.7928,0.806006,0.78066
2,Forest_sanity-3-lin.pth,0.8088,0.823287,0.790642
3,Forest_sanity-3-non-add.pth,0.8704,0.934328,0.508943
4,Forest_sanity-3-non-lin.pth,0.9644,0.975022,0.949664
5,Linear_credit.pth,0.7636,0.780687,0.747063
6,Linear_sanity-3-lin.pth,0.9952,0.99693,0.993879
7,Linear_sanity-3-non-add.pth,0.9988,1,0.995082
8,Linear_sanity-3-non-lin.pth,0.9976,0.999182,0.995922
9,MLP_credit.pth,0.6884,0.682809,0.686688


In [4]:
# Table to Latex 
# Get header
st=''
col=[]
name=[]
df=df.iloc[1:]
for index, row in df.iterrows():
    #print(row['model_name'].split('_'))
    if st is '':
        st += row['model_name'].split('_')[1].replace('.pth','')
    else:
        st += ' & '+row['model_name'].split('_')[1].replace('.pth','')
    col.append(row['model_name'].split('_')[0])
    name.append(row['model_name'].split('_')[1])
st+= ' \\\\ '
col= np.unique(col)
name = np.unique(name)
print(col)
print(name)
for i in range(0,len(col)):
    st += col[i] 
    for j in range(0,len(name)):

        #print()
        print(f'{col[i]}_{name[i]}')
        print(str(df[df['model_name'] ==f'{col[i]}_{name[j]}']['accuracy'].values[0]))
        #print(df[df['model_name'] ==f'{col[i]}_{name[i]}']['accuracy'])
        st += ' & ' + str(df[df['model_name'] ==f'{col[i]}_{name[j]}']['accuracy'].values[0])
    st+=' \\\\ '
print(st)

['Forest' 'Linear' 'MLP']
['credit.pth' 'sanity-3-lin.pth' 'sanity-3-non-add.pth'
 'sanity-3-non-lin.pth']
Forest_credit.pth
0.7928
Forest_credit.pth
0.8088
Forest_credit.pth
0.8704
Forest_credit.pth
0.9644
Linear_sanity-3-lin.pth
0.7636
Linear_sanity-3-lin.pth
0.9952
Linear_sanity-3-lin.pth
0.9988
Linear_sanity-3-lin.pth
0.9976
MLP_sanity-3-non-add.pth
0.6884
MLP_sanity-3-non-add.pth
0.9968
MLP_sanity-3-non-add.pth
0.9908
MLP_sanity-3-non-add.pth
0.998
credit & sanity-3-lin & sanity-3-non-add & sanity-3-non-lin & credit & sanity-3-lin & sanity-3-non-add & sanity-3-non-lin & credit & sanity-3-lin & sanity-3-non-add & sanity-3-non-lin \\ Forest & 0.7928 & 0.8088 & 0.8704 & 0.9644 \\ Linear & 0.7636 & 0.9952 & 0.9988 & 0.9976 \\ MLP & 0.6884 & 0.9968 & 0.9908 & 0.998 \\ 
